In [1]:
# reading the imported file
df = sc.textFile("/FileStore/tables/VS14MORT_txt-b42fa.gz",4)

In [2]:
# view first 5 rows
df.take(5)

Out[ 2 ]: 
[' 1 2101 M1087 432311 4M4 2014U7CN I64 238 070 24 0111I64 01 I64 01 11 100 601',
 ' 1 2101 M1058 371708 4D3 2014U7CN I250214 062 21 0311I250 61I272 62E669 03 I250 E669 I272 01 11 100 601',
 ' 1 7101 F1075 412110 4W2 2014U7CN J449267 086 28 0211J449 61F179 02 J449 F179 01 11 100 601',
 ' 1 6101 M1074 402009 6D1 2014U7CN I48 228 068 22 0311I64 21I48 61F03 03 I48 F03 I64 01 11 100 601',
 ' 1 3101 M1064 381808 4D2 2014U7CN I250214 062 21 0111I250 01 I250 01 11 100 601']

In [3]:
# extract useful information from cryptic-looking df
# parse the unreadable row into something useful 
def extractInformation(row):
    import re
    import numpy as np

    selected_indices = [
         2,4,5,6,7,9,10,11,12,13,14,15,16,17,18,
         19,21,22,23,24,25,27,28,29,30,32,33,34,
         36,37,38,39,40,41,42,43,44,45,46,47,48,
         49,50,51,52,53,54,55,56,58,60,61,62,63,
         64,65,66,67,68,69,70,71,72,73,74,75,76,
         77,78,79,81,82,83,84,85,87,89
    ]

    '''
        Input record schema
        schema: n-m (o) -- xxx
            n - position from
            m - position to
            o - number of characters
            xxx - description
        1. 1-19 (19) -- reserved positions
        2. 20 (1) -- resident status
        3. 21-60 (40) -- reserved positions
        4. 61-62 (2) -- education code (1989 revision)
        5. 63 (1) -- education code (2003 revision)
        6. 64 (1) -- education reporting flag
        7. 65-66 (2) -- month of death
        8. 67-68 (2) -- reserved positions
        9. 69 (1) -- sex
        10. 70 (1) -- age: 1-years, 2-months, 4-days, 5-hours, 6-minutes, 9-not stated
        11. 71-73 (3) -- number of units (years, months etc)
        12. 74 (1) -- age substitution flag (if the age reported in positions 70-74 is calculated using dates of birth and death)
        13. 75-76 (2) -- age recoded into 52 categories
        14. 77-78 (2) -- age recoded into 27 categories
        15. 79-80 (2) -- age recoded into 12 categories
        16. 81-82 (2) -- infant age recoded into 22 categories
        17. 83 (1) -- place of death
        18. 84 (1) -- marital status
        19. 85 (1) -- day of the week of death
        20. 86-101 (16) -- reserved positions
        21. 102-105 (4) -- current year
        22. 106 (1) -- injury at work
        23. 107 (1) -- manner of death
        24. 108 (1) -- manner of disposition
        25. 109 (1) -- autopsy
        26. 110-143 (34) -- reserved positions
        27. 144 (1) -- activity code
        28. 145 (1) -- place of injury
        29. 146-149 (4) -- ICD code
        30. 150-152 (3) -- 358 cause recode
        31. 153 (1) -- reserved position
        32. 154-156 (3) -- 113 cause recode
        33. 157-159 (3) -- 130 infant cause recode
        34. 160-161 (2) -- 39 cause recode
        35. 162 (1) -- reserved position
        36. 163-164 (2) -- number of entity-axis conditions
        37-56. 165-304 (140) -- list of up to 20 conditions
        57. 305-340 (36) -- reserved positions
        58. 341-342 (2) -- number of record axis conditions
        59. 343 (1) -- reserved position
        60-79. 344-443 (100) -- record axis conditions
        80. 444 (1) -- reserve position
        81. 445-446 (2) -- race
        82. 447 (1) -- bridged race flag
        83. 448 (1) -- race imputation flag
        84. 449 (1) -- race recode (3 categories)
        85. 450 (1) -- race recode (5 categories)
        86. 461-483 (33) -- reserved positions
        87. 484-486 (3) -- Hispanic origin
        88. 487 (1) -- reserved
        89. 488 (1) -- Hispanic origin/race recode
     '''

    record_split = re\
        .compile(
            r'([\s]{19})([0-9]{1})([\s]{40})([0-9\s]{2})([0-9\s]{1})([0-9]{1})([0-9]{2})' + 
            r'([\s]{2})([FM]{1})([0-9]{1})([0-9]{3})([0-9\s]{1})([0-9]{2})([0-9]{2})' + 
            r'([0-9]{2})([0-9\s]{2})([0-9]{1})([SMWDU]{1})([0-9]{1})([\s]{16})([0-9]{4})' +
            r'([YNU]{1})([0-9\s]{1})([BCOU]{1})([YNU]{1})([\s]{34})([0-9\s]{1})([0-9\s]{1})' +
            r'([A-Z0-9\s]{4})([0-9]{3})([\s]{1})([0-9\s]{3})([0-9\s]{3})([0-9\s]{2})([\s]{1})' + 
            r'([0-9\s]{2})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([\s]{36})([A-Z0-9\s]{2})([\s]{1})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([\s]{1})([0-9\s]{2})([0-9\s]{1})' + 
            r'([0-9\s]{1})([0-9\s]{1})([0-9\s]{1})([\s]{33})([0-9\s]{3})([0-9\s]{1})([0-9\s]{1})')
    try:
        rs = np.array(record_split.split(row))[selected_indices]
    except:
        rs = np.array(['-99'] * len(selected_indices))
    return rs
#     return record_split.split(row)

In [4]:
# split and convert our dataset
# hand over one element of RDD to exInf()
df_conv = df.map(extractInformation)
df_conv.take(1)

Out[ 4 ]: 
[array(['1', ' ', '2', '1', '01', 'M', '1', '087', ' ', '43', '23', '11',
 ' ', '4', 'M', '4', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I64 ',
 '238', '070', ' ', '24', '01', '11I64 ', ' ', ' ',
 ' ', ' ', ' ', ' ', ' ', ' ',
 ' ', ' ', ' ', ' ', ' ', ' ',
 ' ', ' ', ' ', ' ', ' ', '01',
 'I64 ', ' ', ' ', ' ', ' ', ' ', ' ',
 ' ', ' ', ' ', ' ', ' ', ' ', ' ',
 ' ', ' ', ' ', ' ', ' ', ' ', '01', ' ',
 ' ', '1', '1', '100', '6'], 
 dtype='<U40')]

In [5]:
# create new dataset that will convert year of death into a numeric value
df_2014 = df_conv.map(lambda row: int(row[16]))
df_2014.take(15)

Out[ 6 ]: 
[2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 2014,
 -99,
 2014,
 2014,
 -99,
 2014,
 2014]

In [6]:
# bring more columns: package them into a tuple, dict, or a list
df_2014_2 = df_conv.map(lambda row: (row[16], row[16]))
df_2014_2.take(10)

Out[ 14 ]: 
[('2014', '2014'),
 ('2014', '2014'),
 ('2014', '2014'),
 ('2014', '2014'),
 ('2014', '2014'),
 ('2014', '2014'),
 ('2014', '2014'),
 ('2014', '2014'),
 ('2014', '2014'),
 ('-99', '-99')]

In [7]:
# select elements that fit criteria
# accident deaths in 2014
df_filtered = df_conv.filter(
  lambda row: row[16] == '2014' and row[21] == '0'
  )
df_filtered.count()

Out[ 8 ]: 22

In [8]:
# flattened result instead of list
df_2014_flat = df_conv.flatMap(
  lambda row: (row[16], int(row[16]) + 1)
  )
df_2014_flat.take(10)

Out[ 9 ]: ['2014', 2015, '2014', 2015, '2014', 2015, '2014', 2015, '2014', 2015]

In [9]:
# list of distinct values of dataset
distinct_gender = df_conv.map(
  lambda row: row[5]).distinct()
distinct_gender.collect()

Out[ 15 ]: ['-99', 'M', 'F']

In [10]:
# .sample() returns randomized sample
# first parameter specifies if sampling should be with replacement 
# second parameter defines fraction of data to return 
# third is seed to the pseudo-random numbers generator
fraction = 0.1
df_sample = df_conv.sample(False, fraction, 666)

In [11]:
# size of dataset to confirm 10% sample
print("Original dataset: {0}, sample: {1}".format(df_conv.count(), df_sample.count())) 

Original dataset: 2631171, sample: 263247